In [49]:
import  pandas as pd 
import numpy as np
import matplotlib
import json

In [ ]:
# gdf_with_all_activities = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\df_all_valid_with_activities_complete.csv')

In [ ]:
gdf_with_all_activities, columns = ['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'highway', 'name_en_left', 'ref_old',
       'ref', 'int_ref', 'distance_to_road(metres)', 'road_activity',
       'distance_from_location(m)', 'name_point_shp', 'Activity_point_shp',
       'All_activities_point_shp', 'geometry', 'name_polygon',
       'All_activities_polygon', 'Activity_polygon', 'Activity trace']

In [ ]:
import pandas as pd

# Assuming your DataFrame is named `gdf_with_all_activities`

# Find duplicates based on 'maid' and 'datetime'
# duplicates = gdf_with_all_activities[gdf_with_all_activities.duplicated(subset=['maid', 'datetime'], keep=False)]
duplicates = gdf_with_all_activities_cleaned[gdf_with_all_activities_cleaned.duplicated(subset=['maid', 'datetime'], keep=False)]

# Display the duplicate records
# print(duplicates)   #617087
duplicates_count = duplicates.groupby(['maid', 'datetime']).size().shape[0]
duplicates_count


In [ ]:
import gc
del gdf_with_all_activities_cleaned_1
gc.collect()

In [ ]:
duplicates.head(10)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
columns = ['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance', 'Velocity', 'Activity trace']

# gdf_with_all_activities['tuple_column'] = gdf_with_all_activities[columns].apply(tuple, axis=1)
gdf_with_all_activities_cleaned['tuple_column'] = gdf_with_all_activities_cleaned[columns].apply(tuple, axis=1)

In [ ]:
# Shift displacement values to compare with the next record
duplicates['next_displacement'] = duplicates.groupby('maid')['displacement'].shift(-1)

# Find consecutive records where displacement of the next record is > 0.1 km
consecutive_records = duplicates[(duplicates['next_displacement'] < 0.1) & (duplicates['maid'] == duplicates['maid'].shift(-1)) & (duplicates['datetime'] == duplicates['datetime'].shift(-1))]

# Drop the helper column
# consecutive_records = consecutive_records.drop(columns=['next_displacement'])

# Reset index if needed
consecutive_records = consecutive_records.reset_index(drop=True)

# print(consecutive_records)
consecutive_records

In [ ]:
consecutive_records.columns

In [ ]:
gdf_with_all_activities.columns

In [ ]:
consecutive_records['tuple_column'] = consecutive_records[columns].apply(tuple, axis=1)

# Drop common records from df1 based on these tuples
gdf_with_all_activities_cleaned = gdf_with_all_activities[~gdf_with_all_activities['tuple_column'].isin(consecutive_records['tuple_column'])]


In [ ]:
gdf_with_all_activities_cleaned.shape

In [ ]:
consecutive_records.shape

In [ ]:
# Drop common records from df1
gdf_with_all_activities_cleaned = gdf_with_all_activities[~gdf_with_all_activities.apply(tuple, axis=1).isin(consecutive_records.apply(tuple, axis=1))]

In [ ]:
gdf_with_all_activities_cleaned.shape

In [ ]:
gdf_with_all_activities_cleaned_1.shape

In [ ]:

# gdf_with_all_activities = gdf_with_all_activities.drop_duplicates(subset=['maid', 'datetime'], keep='first')

In [ ]:
# gdf_with_all_activities.shape

In [ ]:
def optimized_function_maid(df, lat_col='latitude', lon_col='longitude', datetime_col='datetime', maid_col='maid'):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(by=[maid_col, datetime_col, lat_col,lon_col])

    # Shift columns to get previous values for each maid
    df['prev_X'] = df.groupby(maid_col)[lon_col].shift(1)
    df['prev_Y'] = df.groupby(maid_col)[lat_col].shift(1)
    df['prev_datetime'] = df.groupby(maid_col)[datetime_col].shift(1)

    # Radius of the Earth in kilometers
    R = 6371

    # Convert latitude and longitude from degrees to radians
    lat1 = np.radians(df[lat_col].values)
    lat2 = np.radians(df['prev_Y'].values)
    dlat = lat2 - lat1
    dlon = np.radians(df['prev_X'].values) - np.radians(df[lon_col].values)

    # Haversine formula for distance calculation
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate consecutive distance in kilometers
    df['consecutive_distance(kms)'] = R * c
    df['cumulative_distance(kms)'] = df.groupby(maid_col)['consecutive_distance(kms)'].cumsum()

    # Calculate consecutive time gap in minutes
    df['consecutive_time_gap(minutes)'] = (df[datetime_col] - df['prev_datetime']).dt.total_seconds() / 60

    # Calculate cumulative time in minutes
    df['cumulative_time(minutes)'] = df.groupby(maid_col)['consecutive_time_gap(minutes)'].cumsum()


    df['Instantaneous_speed(km/h)'] = df['consecutive_distance(kms)'] / (df['consecutive_time_gap(minutes)'] / 60)
    
    # Drop temporary columns
    df = df.drop(columns=['prev_X', 'prev_Y', 'prev_datetime'])

    # Fill NaN values with 0
    df = df.fillna(0)

    return df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
gdf_with_all_activities_cleaned = optimized_function_maid(gdf_with_all_activities_cleaned)
gdf_with_all_activities_cleaned.shape

In [ ]:
gdf_with_all_activities_cleaned['distance']= gdf_with_all_activities_cleaned['cumulative_distance(kms)']
gdf_with_all_activities_cleaned['Velocity']= gdf_with_all_activities_cleaned['Instantaneous_speed(km/h)']
gdf_with_all_activities_cleaned['displacement']= gdf_with_all_activities_cleaned['consecutive_distance(kms)']
gdf_with_all_activities_cleaned.drop(columns=['cumulative_distance(kms)', 'Instantaneous_speed(km/h)','consecutive_distance(kms)'], inplace =True)
gdf_with_all_activities_cleaned.columns

In [ ]:
# gdf_with_all_activities = gdf_with_all_activities_cleaned
# del gdf_with_all_activities_cleaned
gc.collect()

In [ ]:
# Create a boolean column 'equal_check' indicating if 'column1' equals 'column2'
gdf_with_all_activities['equal_check'] = gdf_with_all_activities['Velocity'] == gdf_with_all_activities['Instantaneous_speed(km/h)']
gdf_with_all_activities['equal_check'].unique()

In [ ]:
gdf_with_all_activities.columns

In [ ]:
gdf_with_all_activities.drop(columns = ['consecutive_distance(kms)', 'cumulative_distance(kms)',
       'consecutive_time_gap(minutes)', 'cumulative_time(minutes)',
       'Instantaneous_speed(km/h)', 'equal_check'], inplace=True)

In [ ]:
gdf_with_all_activities['Activity trace'].unique()

In [ ]:
gdf_with_all_activities['Activity trace'].replace(np.nan,'', inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import matplotlib.ticker as ticker

# Function to plot cumulative distance
def plot_activity_trace(df, prefix=''):
    """
    Plots the cumulative distance against cumulative time for each maid in the dataframe.

    Parameters:
    df (pd.DataFrame): DataFrame containing maid, datetime, distance, and path.
    """

    # Check and convert the datetime column if necessary
    if not pd.api.types.is_datetime64_any_dtype(df['datetime']):
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Plotting cumulative distance against cumulative time
    plt.figure(figsize=(18, 6))

    for maid, group in df.groupby('maid'):
        plt.plot(group['datetime'], group['distance'], marker='o', linestyle='-', label=maid)
        
        # Annotate each point with the corresponding path value
        for i, row in group.iterrows():
            plt.annotate(
                row['Activity trace'], 
                (row['datetime'], row['distance']),
                textcoords="offset points", 
                xytext=(5, 5),  # Position of text relative to the point
                ha='left', 
                fontsize=6, 
                alpha=0.7,  # Make the text slightly transparent for clarity
                bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.3')
            )

    plt.xlabel('Date')
    plt.ylabel('Distance (Kms)')

    # Adjusting the legend position and appearance
    plt.legend(
        loc='upper left',
        title='Maid ID',
        fancybox=True,
        framealpha=0.5,
        borderpad=1
    )

    plt.title(f'{prefix}Distance vs Time')

    # Set the x-axis major locator to day and format the dates
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.DayLocator())  # Major ticks for each day
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    # Set minor locator for hourly grid lines
    ax.xaxis.set_minor_locator(mdates.HourLocator(interval=1))  # Minor ticks for 2 hour

    # Set major ticks on the y-axis every 100 units and minor ticks every 50 units
    ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(50))  # Minor ticks for divisions of 50

    plt.xticks(rotation=45)

    # Enable grid for both major and minor ticks
    ax.grid(which='major', linestyle='--', linewidth=1.0)  # Regular grid for major ticks
    ax.grid(which='minor', linestyle=':', linewidth=0.5)   # Regular grid for minor ticks

    # Calculate the start of the day for the first datetime
    first_day_start = df['datetime'].min().normalize()

    # Create a date range with 12-hour frequency starting from first_day_start
    twelve_hour_ticks = pd.date_range(start=first_day_start, end=df['datetime'].max(), freq='12H')

    # Draw thicker lines at each 12-hour interval
    for dt in twelve_hour_ticks:
        ax.axvline(x=dt, linestyle='-', color='gray', linewidth=0.8, alpha=0.5)  # Thicker line for 12-hour grid

    plt.tight_layout()
    plt.show()


In [ ]:
# gdf_with_all_activities= pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\df_all_valid_with_activities_info_compact.csv')
gdf_with_all_activities = gdf_with_all_activities_cleaned

In [ ]:
# gdf_with_all_activities['Activity trace'].replace(0, np.nan, inplace=True)
gdf_with_all_activities['Activity trace']

In [ ]:
import pandas as pd

# Calculate the total number of distinct values of 'Activity Trace' per maid
distinct_values_count = gdf_with_all_activities.groupby('maid')['Activity trace'].nunique()

# Calculate the total count for 'Activity Trace' having a value other than ''
non_empty_count = gdf_with_all_activities[gdf_with_all_activities['Activity trace'] != ''].groupby('maid')['Activity trace'].count()

# Combine the results into a new DataFrame
maid_activity_trace_stats = pd.DataFrame({
    'maid': distinct_values_count.index,
    'distinct_activity_trace_count': distinct_values_count.values,
    'non_empty_activity_trace_count': non_empty_count.reindex(distinct_values_count.index, fill_value=0).values
})
maid_activity_trace_stats = maid_activity_trace_stats.sort_values(by=['non_empty_activity_trace_count', ], ascending=False)
# Reset index for better readability
maid_activity_trace_stats.reset_index(drop=True, inplace=True)

maid_activity_trace_stats

In [2]:
import pandas as pd

In [14]:
df = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_with_states_districts_new_without_duplicates_and_cleaned.csv')

In [15]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [16]:
df = df.sort_values(by=['maid', 'datetime', 'latitude', 'longitude'])
df['origin_state'] = df.groupby('maid')['state'].transform('first')
df['origin_district'] = df.groupby('maid')['district'].transform('first')

In [19]:
df.shape  # = df[df['state']=='Uttarakhand']

(8266415, 18)

In [18]:
df.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Trajectory 2\df_with_Uttarakhand_records_only_for_316668_ids.csv', index=False)

In [20]:
# Group by 'maid' and 'datetime' (date only), then count records
count_per_day_per_id = df.groupby(['maid', df['datetime'].dt.date]).size().reset_index(name='count')

# Pivot the table to have dates as columns and IDs as rows
pivot_df = count_per_day_per_id.pivot(index='maid', columns='datetime', values='count').fillna(0)

# Select the date columns for calculation
date_columns = pivot_df.columns

# Add summary statistics for the date columns
pivot_df['Total'] = pivot_df[date_columns].sum(axis=1)
pivot_df['Max'] = pivot_df[date_columns].max(axis=1)
pivot_df['Min'] = pivot_df[date_columns].min(axis=1)
pivot_df['Mean'] = pivot_df[date_columns].mean(axis=1)
pivot_df['Std. Deviation'] = pivot_df[date_columns].std(axis=1)
pivot_df.head()


datetime                              2022-09-01  2022-09-02  2022-09-03  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773         8.0         1.0         0.0   
00004eb1-6900-419c-b4be-0ec37b789989         2.0         1.0         0.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         2.0         2.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         0.0   

datetime                              2022-09-04  2022-09-05  2022-09-06  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773         2.0         0.0         0.0   
00004eb1-6900-419c-b4be-0ec37b789989         2.0         0.0         0.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         2.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         4.0         0.0         1.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         0.0   

datetime                              2022-09-07  2022-09-08  2022-09-09  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773         0.0         2.0         1.0   
00004eb1-6900-419c-b4be-0ec37b789989         0.0         0.0         2.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         2.0         2.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         0.0   

datetime                              2022-09-10  ...  2022-09-26  2022-09-27  \
maid                                              ...                           
000010b7-6bd5-4296-9235-2a824209b773         0.0  ...         0.0         0.0   
00004eb1-6900-419c-b4be-0ec37b789989         0.0  ...         4.0         1.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0  ...         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         4.0  ...         0.0         0.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0  ...         0.0         0.0   

datetime                              2022-09-28  2022-09-29  2022-09-30  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773         0.0         0.0         0.0   
00004eb1-6900-419c-b4be-0ec37b789989         1.0         0.0         1.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         0.0         0.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         0.0   

datetime                              Total   Max  Min      Mean  \
maid                                                               
000010b7-6bd5-4296-9235-2a824209b773   68.0  21.0  0.0  2.266667   
00004eb1-6900-419c-b4be-0ec37b789989   67.0  11.0  0.0  2.233333   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d    2.0   2.0  0.0  0.066667   
00007182-8140-450f-b91d-b4522543e0f6   45.0  10.0  0.0  1.500000   
000097a4-0ed7-47a8-b01f-b2c727cbc993    1.0   1.0  0.0  0.033333   

datetime                              Std. Deviation  
maid                                                  
000010b7-6bd5-4296-9235-2a824209b773        4.600849  
00004eb1-6900-419c-b4be-0ec37b789989        2.932380  
00005df5-30a2-4d15-ba7f-35bcd02f7f9d        0.365148  
00007182-8140-450f-b91d-b4522543e0f6        2.501724  
000097a4-0ed7-47a8-b01f-b2c727cbc993        0.182574  

[5 rows x 35 columns]

In [ ]:
# Group by 'maid' and 'datetime' (date only), then count records
count_per_day_per_id = gdf_with_all_activities.groupby(['maid', gdf_with_all_activities['datetime'].dt.date]).size().reset_index(name='count')

# Pivot the table to have dates as columns and IDs as rows
pivot_df = count_per_day_per_id.pivot(index='maid', columns='datetime', values='count').fillna(0)

# Select the date columns for calculation
date_columns = pivot_df.columns

# Add summary statistics for the date columns
pivot_df['Total'] = pivot_df[date_columns].sum(axis=1)
pivot_df['Max'] = pivot_df[date_columns].max(axis=1)
pivot_df['Min'] = pivot_df[date_columns].min(axis=1)
pivot_df['Mean'] = pivot_df[date_columns].mean(axis=1)
pivot_df['Std. Deviation'] = pivot_df[date_columns].std(axis=1)
pivot_df.head()


In [21]:
pivot_df.shape

(316668, 35)

In [26]:
# pivot_df.to_excel(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\distirbution of UK records datewise.xlsx')
pivot_df.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Trajectory 2\distirbution of UK records datewise.csv')

In [25]:
pivot_df = pivot_df.sort_values(by=['Total'], ascending= False)
valid_ids_with_reasonable_distribution = pivot_df[pivot_df['Total']>=60].index.tolist()

In [ ]:
pivot_df.columns

In [40]:
import pandas as pd

# Ensure column names are strings
pivot_df.columns = pivot_df.columns.astype(str)

# Import pandas if not already imported
import pandas as pd

# Step 1: Extract date columns only
date_columns = [col for col in pivot_df.columns if '2022' in col]
print(f'date_columns: {date_columns}')
# Step 2: Initialize a list to store indices where condition is met
selected_maids = []

# Step 3: Loop through the DataFrame to check for the condition
for idx, row in pivot_df.iterrows():
    # Create a list of the date values for the current row
    date_values = row[date_columns].values
    
    # Check for three consecutive days with more than 10 points
    for i in range(len(date_values) - 2):
        if all(date_values[i:i+3] > 4):
            selected_maids.append(idx)
            break

# Output the selected indices
print(selected_maids)

date_columns: ['2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24', '2022-09-25', '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30']
['4ba85554-7b66-43d1-82b8-64b8d0b1e76c', 'eecc13d2-e18e-4258-bb38-a665f33d7407', '29e36dc3-78a4-4aef-87f6-58ca15069857', '720a8a2a-9f79-42d5-83e4-1d90f252480e', '728450b1-610d-4b7b-98f3-503127564e62', 'cc89c136-1f5d-445a-b451-a7856f47e939', '0d17bac0-bfaa-4cac-92de-ac0181cc5760', '8c1e5ecf-77d2-4063-8bd3-2714c1ccbd29', '68c03496-4278-4f92-8398-822ef700a80e', '3ea85b7d-bdaf-4b72-bc15-a13848c1863c', '0977a026-e49e-4337-aeb7-c387b14b355d', '99203cc9-0df4-423d-907e-5a45ef73f3eb', '9495b232-6cbe-432a-8488-1bd8804abe40', '7384fb83-5dc7-4e71-a5f2-70208bab5415', '093

In [41]:
print(len(selected_maids))

37157


In [46]:
import json
# file_path = r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\IDs_with_more_than_10_points_for_atleast_3_consecutive_days.json'
file_path = r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Trajectory 2\valid_distribution\ids_with_4_points_3_days_37157.json'
    
with open(file_path, 'w') as json_file:
    json.dump(selected_maids, json_file)

In [37]:
# len(valid_ids_with_reasonable_distribution)

40598

In [45]:
file_path = r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Trajectory 2\valid_distribution\ids_with_4_points_3_days_30743.json'
    
with open(file_path, 'r') as json_file:
    selected_maids = json.load(json_file)
len(selected_maids)

37157

In [47]:
# filtered_dataframe_stage_1 = gdf_with_all_activities[gdf_with_all_activities['maid'].isin(set(selected_maids))]
# filtered_dataframe_stage_1.shape
filtered_dataframe_stage_1 = df[df['maid'].isin(set(selected_maids))]
filtered_dataframe_stage_1.shape

(5780982, 18)

In [48]:
filtered_dataframe_stage_1.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\UK_records_37157_IDs.csv', index=False)

In [ ]:
import pandas as pd

# Step 1: Calculate the total number of distinct values of 'Activity Trace' per maid
distinct_values_count = filtered_dataframe_stage_1.groupby('maid')['Activity trace'].nunique()

# Step 2: Calculate the total count for 'Activity Trace' having value other than '' per maid
non_empty_count = filtered_dataframe_stage_1[filtered_dataframe_stage_1['Activity trace'] != ''].groupby('maid')['Activity trace'].count()

# Create a DataFrame with these metrics
metrics_df = pd.DataFrame({
    'distinct_values': distinct_values_count,
    'non_empty_count': non_empty_count
}).fillna(0)  # Fill NaNs with 0 for those maids that might not have non-empty values

# Step 3: Sort the DataFrame based on the metrics
# Priority 1: Total number of distinct values of 'Activity Trace'
# Priority 2: Total count for 'Activity Trace' having value other than ''
metrics_df_sorted = metrics_df.sort_values(by=['distinct_values', 'non_empty_count'], ascending=[False, False])

# Reorder the original DataFrame based on the sorted metrics
# Map sorted indices back to the original DataFrame
# gdf_with_all_activities_sorted = gdf_with_all_activities.set_index('maid').reindex(metrics_df_sorted.index).reset_index()

# # # If you want to include the sorting in the final DataFrame
# gdf_with_all_activities_sorted = gdf_with_all_activities_sorted.sort_values(by=['maid'])
metrics_df_sorted

In [ ]:
len(metrics_df_sorted[metrics_df_sorted['non_empty_count']==0].index)

In [ ]:
ids_with_no_activity = metrics_df_sorted[metrics_df_sorted['non_empty_count']==0].index

filtered_dataframe_stage_2 = filtered_dataframe_stage_1[~filtered_dataframe_stage_1['maid'].isin(ids_with_no_activity)]

In [ ]:
filtered_dataframe_stage_2['maid'].nunique()

#### Integrate railway shape file

In [50]:
import geopandas as gpd
from shapely.geometry import Point
uk_railways = gpd.read_file(r"C:\Users\User\Downloads\mobility_traces_complete_data\shape file for roads and railway network\QuickOSM generated\Uttarakhand_railways.shp")
# uk_railways = uk_roads[['highway','ref_old','ref','geometry']]
uk_railways = uk_railways[['name', 'geometry']]
# uk_railways['name'].unique().tolist()

In [61]:
uk_railways

name                                           geometry
0    None  LINESTRING (8715235.072 3517709.831, 8715201.4...
1    None  LINESTRING (8707403.022 3510023.137, 8707296.3...
2    None  LINESTRING (8707528.992 3510185.156, 8707403.0...
3    None  LINESTRING (8686729.824 3477516.344, 8687808.1...
4    None  LINESTRING (8645198.025 3493255.316, 8645480.3...
..    ...                                                ...
421  None  LINESTRING (8810707.443 3395759.667, 8810734.2...
422  None  LINESTRING (8810786.202 3395732.388, 8810839.3...
423  None  LINESTRING (8714849.539 3517239.411, 8714817.5...
424  None  LINESTRING (8715104.082 3517561.421, 8715133.6...
425  None  LINESTRING (8714778.095 3517220.778, 8714821.0...

[426 rows x 2 columns]

In [ ]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\df_IDs_with_valid_distribution_and_atleast_one_Activity.csv', index=False)

In [55]:
import gc
del filtered_dataframe_stage_1

gc.collect()

NameError: name 'filtered_dataframe_stage_1' is not defined

In [56]:
filtered_dataframe_stage_2 = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\step_3_UK_records_with_roads.csv')

In [57]:
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.reset_index(drop=True)

In [58]:
filtered_dataframe_stage_2['geometry']

0           POINT (8759619.411031904 3531473.777856918)
1           POINT (8759619.411031904 3531473.777856918)
2          POINT (8760320.723823901 3533122.8232502528)
3          POINT (8760320.723823901 3533122.8232502528)
4           POINT (8760020.16119876 3533161.4753259206)
                               ...                     
5780977    POINT (8801798.366093475 3597915.2300974256)
5780978    POINT (8801787.234144397 3597941.1312578265)
5780979    POINT (8801787.234144397 3597941.1312578265)
5780980    POINT (8801776.102195317 3597941.1312578265)
5780981     POINT (8801720.44244992 3598200.1458181906)
Name: geometry, Length: 5780982, dtype: object

In [59]:
filtered_dataframe_stage_2.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'Nearest airport',
       'Distance to airport (km)', 'origin_state', 'origin_district', 'date',
       'time', 'y', 'x', 'position_group', 'time_group', 'geometry',
       'index_right', 'highway', 'ref_old', 'ref', 'distance_to_road(m)'],
      dtype='object')

In [ ]:
filtered_dataframe_stage_2.drop(columns = ['tuple_column', 'consecutive_time_gap(minutes)', 'cumulative_time(minutes)'], inplace=True)

In [64]:
from shapely.geometry import Point
filtered_dataframe_stage_2['geometry'] = filtered_dataframe_stage_2.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

filtered_dataframe_stage_2 = gpd.GeoDataFrame(filtered_dataframe_stage_2, geometry='geometry')
filtered_dataframe_stage_2.set_crs("EPSG:4326", inplace=True, allow_override=True)
# # Print the CRS information
# print(gdf.crs)

maid             datetime  latitude  \
0        00007182-8140-450f-b91d-b4522543e0f6  2022-09-02 13:05:15   30.2170   
1        00007182-8140-450f-b91d-b4522543e0f6  2022-09-02 13:05:21   30.2170   
2        00007182-8140-450f-b91d-b4522543e0f6  2022-09-03 16:44:33   30.2298   
3        00007182-8140-450f-b91d-b4522543e0f6  2022-09-03 18:06:09   30.2298   
4        00007182-8140-450f-b91d-b4522543e0f6  2022-09-04 02:07:37   30.2301   
...                                       ...                  ...       ...   
5780977  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d  2022-09-16 03:46:18   30.7314   
5780978  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d  2022-09-16 06:38:33   30.7316   
5780979  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d  2022-09-16 06:42:57   30.7316   
5780980  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d  2022-09-16 16:40:41   30.7316   
5780981  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d  2022-09-16 16:43:31   30.7336   

         longitude  displacement     distance     Velocity        state  \
0          78.6890    226.448884   226.448884    49.687084  Uttarakhand   
1          78.6890      0.000000   226.448884     0.000000  Uttarakhand   
2          78.6953    225.552682   676.069836     9.643812  Uttarakhand   
3          78.6953      0.000000   676.069836     0.000000  Uttarakhand   
4          78.6926    225.414322  1131.839375  1539.832183  Uttarakhand   
...            ...           ...          ...          ...          ...   
5780977    79.0679    293.813191  7795.669911    20.190263  Uttarakhand   
5780978    79.0678      0.024206  7795.694117     0.008432  Uttarakhand   
5780979    79.0678      0.000000  7795.694117     0.000000  Uttarakhand   
5780980    79.0677      0.009558  7795.703675     0.000959  Uttarakhand   
5780981    79.0672      0.227467  7795.931142     4.816941  Uttarakhand   

              district Nearest airport  ...             y             x  \
0        Pauri Garhwal               0  ...  8.759619e+06  3.531474e+06   
1        Pauri Garhwal        DEHRADUN  ...  8.759619e+06  3.531474e+06   
2        Pauri Garhwal               0  ...  8.760321e+06  3.533123e+06   
3        Pauri Garhwal               0  ...  8.760321e+06  3.533123e+06   
4        Pauri Garhwal        DEHRADUN  ...  8.760020e+06  3.533161e+06   
...                ...             ...  ...           ...           ...   
5780977    Rudraprayag               0  ...  8.801798e+06  3.597915e+06   
5780978    Rudraprayag               0  ...  8.801787e+06  3.597941e+06   
5780979    Rudraprayag               0  ...  8.801787e+06  3.597941e+06   
5780980    Rudraprayag               0  ...  8.801776e+06  3.597941e+06   
5780981    Rudraprayag               0  ...  8.801720e+06  3.598200e+06   

        position_group time_group                 geometry  index_right  \
0           x_763y_493      t_157    POINT (78.689 30.217)         6184   
1           x_763y_493      t_157    POINT (78.689 30.217)         6184   
2           x_769y_496      t_200  POINT (78.6953 30.2298)         6183   
3           x_769y_496      t_217  POINT (78.6953 30.2298)         6183   
4           x_770y_495       t_25  POINT (78.6926 30.2301)         6183   
...                ...        ...                      ...          ...   
5780977    x_1029y_662       t_45  POINT (79.0679 30.7314)          959   
5780978    x_1029y_662       t_79  POINT (79.0678 30.7316)          959   
5780979    x_1029y_662       t_80  POINT (79.0678 30.7316)          959   
5780980    x_1029y_662      t_200  POINT (79.0677 30.7316)          959   
5780981    x_1030y_662      t_200  POINT (79.0672 30.7336)          959   

          highway ref_old    ref distance_to_road(m)  
0        tertiary     NaN    NaN           98.494476  
1        tertiary     NaN    NaN           98.494476  
2        tertiary     NaN    NaN           15.103232  
3        tertiary     NaN    NaN           15.103232  
4        tertiary     NaN    NaN          111.879120  
...           ...     ...    ...               

In [170]:
# filtered_dataframe_stage_2.drop(columns=['train_activity','name_left',	'dist_to_rail(m)',	'index_right'	,'name_right'], inplace=True)

In [65]:
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.to_crs("EPSG:3857")
uk_railways = uk_railways.to_crs("EPSG:3857")

In [73]:
filtered_dataframe_stage_2.drop(columns = ['index_right'], inplace=True)
# uk_railways.columns

In [ ]:
filtered_dataframe_stage_2 = gpd.sjoin_nearest(filtered_dataframe_stage_2, uk_railways, how='left', distance_col='dist_to_rail(m)')
filtered_dataframe_stage_2.reset_index(drop=True)

In [76]:
filtered_dataframe_stage_2.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'Nearest airport',
       'Distance to airport (km)', 'origin_state', 'origin_district', 'date',
       'time', 'y', 'x', 'position_group', 'time_group', 'geometry', 'highway',
       'ref_old', 'ref', 'distance_to_road(m)', 'name', 'dist_to_rail(m)'],
      dtype='object')

In [77]:
# Sort the DataFrame by the specified columns
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.sort_values(by=['maid', 'datetime', 'latitude', 'longitude', 'dist_to_rail(m)'])

# Drop duplicates, keeping only the first (smallest 'distance_to_road(m)') record for each combination of maid, datetime, latitude, and longitude
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'], keep='first')

# gdf_intrastate_unique now contains only the records with the minimum 'distance_to_road(m)' for each combination
filtered_dataframe_stage_2.shape

(5780982, 26)

In [78]:
filtered_dataframe_stage_2.rename(columns = {'name':'name_rail'}, inplace=True)

In [79]:
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.sort_values(by = ['maid', 'datetime', 'latitude','longitude', 'dist_to_rail(m)'])

In [80]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\step_4_UK_records_with_rails.csv')

In [97]:
filtered_dataframe_stage_2 = filtered_dataframe_stage_2[['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'Nearest airport',
       'Distance to airport (km)', 'origin_state', 'origin_district', 'date',
       'time', 'y', 'x', 'position_group', 'time_group', 'geometry', 'highway',
       'ref_old', 'ref_left', 'distance_to_road(m)', 'name_rail',
       'dist_to_rail(m)']]

In [98]:
filtered_dataframe_stage_2['datetime']= pd.to_datetime(filtered_dataframe_stage_2['datetime'])

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


#### Integrating with airport boundary files

In [100]:
uk_airports = gpd.read_file(r"C:\Users\User\Downloads\mobility_traces_complete_data\shape file for roads and railway network\QuickOSM generated\Uttarakhand_airports_boundary.shp")
uk_airports = uk_airports[['name', 'geometry']]


In [101]:
uk_airports.columns

Index(['name', 'geometry'], dtype='object')

In [102]:
uk_airports

name  \
0                    Pithoragarh   
1   Dehradun Jolly Grant Airport   
2              Pantnagar Airport   
3                           None   
4                           None   
..                           ...   
82                          None   
83                          None   
84                          None   
85                          None   
86                          None   

                                             geometry  
0   POLYGON ((80.23944 29.59266, 80.23462 29.5981,...  
1   POLYGON ((78.19408 30.19743, 78.19412 30.19723...  
2   POLYGON ((79.46291 29.033, 79.4637 29.03643, 7...  
3   POLYGON ((78.21247 30.01463, 78.21297 30.01464...  
4   POLYGON ((79.49466 30.73498, 79.49479 30.735, ...  
..                                                ...  
82  POLYGON ((79.06509 30.70688, 79.06509 30.7069,...  
83  POLYGON ((78.32248 30.58393, 78.32245 30.58396...  
84  POLYGON ((78.32314 30.58412, 78.32311 30.58414...  
85  POLYGON ((79.08024 30.53927, 79.0803 30.5393, ...  
86  POLYGON ((78.45729 30.4066, 78.45697 30.40719,...  

[87 rows x 2 columns]

In [ ]:
uk_airports['name'].fillna('Name not available')

In [104]:
uk_airports.to_crs("EPSG:3857")

name  \
0                    Pithoragarh   
1   Dehradun Jolly Grant Airport   
2              Pantnagar Airport   
3                           None   
4                           None   
..                           ...   
82                          None   
83                          None   
84                          None   
85                          None   
86                          None   

                                             geometry  
0   POLYGON ((8932213.736 3451296.363, 8931676.586...  
1   POLYGON ((8704525.091 3528952.454, 8704529.655...  
2   POLYGON ((8845770.244 3379846.561, 8845858.376...  
3   POLYGON ((8706572.245 3505430.494, 8706627.916...  
4   POLYGON ((8849304.872 3598378.442, 8849319.588...  
..                                                ...  
82  POLYGON ((8801485.915 3594740.064, 8801485.347...  
83  POLYGON ((8718818.524 3578831.747, 8718815.764...  
84  POLYGON ((8718891.895 3578855.708, 8718889.146...  
85  POLYGON ((8803171.971 3573058.132, 8803178.216...  
86  POLYGON ((8733825.472 3555922.014, 8733790.361...  

[87 rows x 2 columns]

In [95]:
filtered_dataframe_stage_2.shape

(5780982, 55)

In [128]:
# filtered_dataframe_stage_2.drop(columns = ['name_left','index_right', 'name_right' ], inplace = True)

In [92]:
filtered_dataframe_stage_2.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'Nearest airport',
       'Distance to airport (km)', 'origin_state', 'origin_district', 'date',
       'time', 'y', 'x', 'position_group', 'time_group', 'geometry', 'highway',
       'ref_old', 'ref', 'distance_to_road(m)', 'name_rail',
       'dist_to_rail(m)'],
      dtype='object')

In [151]:
filtered_dataframe_stage_2.rename(columns= {'name':'name_airport_point'}, inplace=True)

In [109]:
# filtered_dataframe_stage_2.drop(columns = [ 'index_right', 'name'], inplace=True)

In [110]:
# Reproject uk_airports to match the CRS of filtered_dataframe_stage_2
uk_airports = uk_airports.to_crs(filtered_dataframe_stage_2.crs)

In [111]:
# Perform spatial join
filtered_dataframe_stage_2 = gpd.sjoin(filtered_dataframe_stage_2, uk_airports, how="left", predicate='within')

# Display the result
filtered_dataframe_stage_2

maid            datetime  latitude  \
0        00007182-8140-450f-b91d-b4522543e0f6 2022-09-02 13:05:15   30.2170   
1        00007182-8140-450f-b91d-b4522543e0f6 2022-09-02 13:05:21   30.2170   
2        00007182-8140-450f-b91d-b4522543e0f6 2022-09-03 16:44:33   30.2298   
3        00007182-8140-450f-b91d-b4522543e0f6 2022-09-03 18:06:09   30.2298   
4        00007182-8140-450f-b91d-b4522543e0f6 2022-09-04 02:07:37   30.2301   
...                                       ...                 ...       ...   
5780977  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 03:46:18   30.7314   
5780978  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:38:33   30.7316   
5780979  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:42:57   30.7316   
5780980  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:40:41   30.7316   
5780981  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:43:31   30.7336   

         longitude  displacement     distance     Velocity        state  \
0          78.6890    226.448884   226.448884    49.687084  Uttarakhand   
1          78.6890      0.000000   226.448884     0.000000  Uttarakhand   
2          78.6953    225.552682   676.069836     9.643812  Uttarakhand   
3          78.6953      0.000000   676.069836     0.000000  Uttarakhand   
4          78.6926    225.414322  1131.839375  1539.832183  Uttarakhand   
...            ...           ...          ...          ...          ...   
5780977    79.0679    293.813191  7795.669911    20.190263  Uttarakhand   
5780978    79.0678      0.024206  7795.694117     0.008432  Uttarakhand   
5780979    79.0678      0.000000  7795.694117     0.000000  Uttarakhand   
5780980    79.0677      0.009558  7795.703675     0.000959  Uttarakhand   
5780981    79.0672      0.227467  7795.931142     4.816941  Uttarakhand   

              district Nearest airport  ...  time_group  \
0        Pauri Garhwal               0  ...       t_157   
1        Pauri Garhwal        DEHRADUN  ...       t_157   
2        Pauri Garhwal               0  ...       t_200   
3        Pauri Garhwal               0  ...       t_217   
4        Pauri Garhwal        DEHRADUN  ...        t_25   
...                ...             ...  ...         ...   
5780977    Rudraprayag               0  ...        t_45   
5780978    Rudraprayag               0  ...        t_79   
5780979    Rudraprayag               0  ...        t_80   
5780980    Rudraprayag               0  ...       t_200   
5780981    Rudraprayag               0  ...       t_200   

                                geometry   highway ref_old ref_left  \
0        POINT (8759619.411 3531473.778)  tertiary     NaN      NaN   
1        POINT (8759619.411 3531473.778)  tertiary     NaN      NaN   
2        POINT (8760320.724 3533122.823)  tertiary     NaN      NaN   
3        POINT (8760320.724 3533122.823)  tertiary     NaN      NaN   
4        POINT (8760020.161 3533161.475)  tertiary     NaN      NaN   
...                                  ...       ...     ...      ...   
5780977   POINT (8801798.366 3597915.23)     trunk   NH109    NH107   
5780978  POINT (8801787.234 3597941.131)     trunk   NH109    NH107   
5780979  POINT (8801787.234 3597941.131)     trunk   NH109    NH107   
5780980  POINT (8801776.102 3597941.131)     trunk   NH109    NH107   
5780981  POINT (8801720.442 3598200.146)     trunk   NH109    NH107   

         distance_to_road(m)  name_rail dist_to_rail(m) index_right name  
0                  98.494476       None    46469.514390         NaN  NaN  
1                  98.494476       None    46469.514390         NaN  NaN  
2                  15.103232       None    47647.416646         NaN  NaN  
3                  15.103232       None    47647.416646         NaN  NaN  
4                 111.879120       None    47375.705900         NaN  NaN  
...                      ...        ...             ...         ...  ...  
5780977         11234.934581       None   118008.939831         NaN  NaN  
5780978         11253.820135 

In [112]:
filtered_dataframe_stage_2.rename(columns={'name':'name_airport_boundary'}, inplace= True)

In [113]:
filtered_dataframe_stage_2.drop(columns = ['index_right'], inplace=True)

In [114]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\step_5_UK_records_with_airport_boundaries.csv')

#### Integrating airport point files

In [115]:
uk_airports_points = gpd.read_file(r"C:\Users\User\Downloads\mobility_traces_complete_data\shape file for roads and railway network\QuickOSM generated\Uttarakhand_airports_points.shp")
uk_airports_points = uk_airports_points[['name', 'geometry']]

In [136]:
# filtered_dataframe_stage_2.drop(columns='index_right', inplace=True)

In [116]:
# uk_airports_points= uk_airports_points.to_crs("EPSG:3857")
uk_airports_points = uk_airports_points.to_crs(filtered_dataframe_stage_2.crs)

In [117]:
uk_airports_points.rename(columns={'name':'name_airport_point'}, inplace=True)

In [118]:
filtered_dataframe_stage_2 = gpd.sjoin_nearest(filtered_dataframe_stage_2, uk_airports_points, how='left', distance_col='dist_to_airport(m)')
filtered_dataframe_stage_2.reset_index(drop=True)

maid            datetime  latitude  \
0        00007182-8140-450f-b91d-b4522543e0f6 2022-09-02 13:05:15   30.2170   
1        00007182-8140-450f-b91d-b4522543e0f6 2022-09-02 13:05:21   30.2170   
2        00007182-8140-450f-b91d-b4522543e0f6 2022-09-03 16:44:33   30.2298   
3        00007182-8140-450f-b91d-b4522543e0f6 2022-09-03 18:06:09   30.2298   
4        00007182-8140-450f-b91d-b4522543e0f6 2022-09-04 02:07:37   30.2301   
...                                       ...                 ...       ...   
5781001  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 03:46:18   30.7314   
5781002  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:38:33   30.7316   
5781003  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:42:57   30.7316   
5781004  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:40:41   30.7316   
5781005  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:43:31   30.7336   

         longitude  displacement     distance     Velocity        state  \
0          78.6890    226.448884   226.448884    49.687084  Uttarakhand   
1          78.6890      0.000000   226.448884     0.000000  Uttarakhand   
2          78.6953    225.552682   676.069836     9.643812  Uttarakhand   
3          78.6953      0.000000   676.069836     0.000000  Uttarakhand   
4          78.6926    225.414322  1131.839375  1539.832183  Uttarakhand   
...            ...           ...          ...          ...          ...   
5781001    79.0679    293.813191  7795.669911    20.190263  Uttarakhand   
5781002    79.0678      0.024206  7795.694117     0.008432  Uttarakhand   
5781003    79.0678      0.000000  7795.694117     0.000000  Uttarakhand   
5781004    79.0677      0.009558  7795.703675     0.000959  Uttarakhand   
5781005    79.0672      0.227467  7795.931142     4.816941  Uttarakhand   

              district Nearest airport  ...   highway ref_old ref_left  \
0        Pauri Garhwal               0  ...  tertiary     NaN      NaN   
1        Pauri Garhwal        DEHRADUN  ...  tertiary     NaN      NaN   
2        Pauri Garhwal               0  ...  tertiary     NaN      NaN   
3        Pauri Garhwal               0  ...  tertiary     NaN      NaN   
4        Pauri Garhwal        DEHRADUN  ...  tertiary     NaN      NaN   
...                ...             ...  ...       ...     ...      ...   
5781001    Rudraprayag               0  ...     trunk   NH109    NH107   
5781002    Rudraprayag               0  ...     trunk   NH109    NH107   
5781003    Rudraprayag               0  ...     trunk   NH109    NH107   
5781004    Rudraprayag               0  ...     trunk   NH109    NH107   
5781005    Rudraprayag               0  ...     trunk   NH109    NH107   

        distance_to_road(m) name_rail  dist_to_rail(m)  name_airport_boundary  \
0                 98.494476      None     46469.514390                    NaN   
1                 98.494476      None     46469.514390                    NaN   
2                 15.103232      None     47647.416646                    NaN   
3                 15.103232      None     47647.416646                    NaN   
4                111.879120      None     47375.705900                    NaN   
...                     ...       ...              ...                    ...   
5781001        11234.934581      None    118008.939831                    NaN   
5781002        11253.820135      None    118018.381044                    NaN   
5781003        11253.820135      None    118018.381044                    NaN   
5781004        11249.185232      None    118010.217353                    NaN   
5781005        11462.671774      None    118145.715253                    NaN   

        index_right name_airport_point dist_to_airport(m)  
0                10               None       55344.697590  
1                10               None       55344.697590  
2                10               None       56161.107468  
3                10               None       56161.107468  
4                10          

In [124]:
# filtered_dataframe_stage_2.drop(columns=['index_right'], inplace=True)
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.sort_values(by=['maid', 'datetime', 'latitude', 'longitude', 'dist_to_airport(m)'])

In [128]:
# Drop duplicates, keeping only the first (smallest 'distance_to_road(m)') record for each combination of maid, datetime, latitude, and longitude
filtered_dataframe_stage_2 = filtered_dataframe_stage_2.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'], keep='first')

# gdf_intrastate_unique now contains only the records with the minimum 'distance_to_road(m)' for each combination
filtered_dataframe_stage_2.shape

(5780982, 29)

In [127]:
filtered_dataframe_stage_2['name_airport_boundary'].unique().tolist()

[nan,
 'Dehradun Jolly Grant Airport',
 'Gauchar Airport',
 None,
 'Himalaya Heli Service',
 'Pantnagar Airport',
 'Pithoragarh',
 'Clement Town Helipad',
 'Uttarakhand Civil Aviation Development Authority',
 'Maa Ganga Airport']

In [129]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\step_6_UK_records_with_airport_boundaries_and_point.csv', index=False)

### Marking respective activity

In [190]:
filtered_dataframe_stage_2['train_activity'] = filtered_dataframe_stage_2['dist_to_rail(m)'].apply(lambda x: 'Train'  if x<25 else None)

In [ ]:
filtered_dataframe_stage_2[filtered_dataframe_stage_2['dist_to_rail(m)']<25].count() # 14036
filtered_dataframe_stage_2[filtered_dataframe_stage_2['train_activity'].notna()]

In [193]:
filtered_dataframe_stage_2['train_activity'] = filtered_dataframe_stage_2.apply(
    lambda row: row['train_activity'] + ' (' + row['name_rail'] + ')' if row['train_activity'] == 'Train' and row['name_rail'] is not None else row['train_activity'],
    axis=1
)

In [194]:
filtered_dataframe_stage_2.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'district', 'highway', 'name_en_left', 'ref_old',
       'ref', 'int_ref', 'distance_to_road(metres)', 'road_activity',
       'distance_from_location(m)', 'name_point_shp', 'Activity_point_shp',
       'All_activities_point_shp', 'geometry', 'name_polygon',
       'All_activities_polygon', 'Activity_polygon', 'Activity trace',
       'name_airport_boundary', 'name_rail', 'dist_to_rail(m)', 'index_right',
       'name_airport_point', 'dist_to_airport(m)', 'train_activity'],
      dtype='object')

In [195]:
filtered_dataframe_stage_2['flight_activity'] = filtered_dataframe_stage_2['dist_to_airport(m)'].apply(lambda x: 'Flight'  if x<1000 else None)

In [196]:
filtered_dataframe_stage_2['flight_activity'] = filtered_dataframe_stage_2.apply(
    lambda row: row['flight_activity'] + ' (' + row['name_airport_point'] + ')' if row['flight_activity'] == 'Flight' and row['name_airport_point'] is not None else row['flight_activity'],
    axis=1
)

In [199]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\df_IDs_with_valid_dist_and_atleast_one_Activity_witg_train_and_rail.csv')

In [ ]:
x = gdf_with_all_activities[gdf_with_all_activities['maid']=='000010b7-6bd5-4296-9235-2a824209b773']
plot_activity_trace(x, prefix='Activity Trace')

#### Travel mode determination

In [131]:
import pandas as pd
from tqdm import tqdm

# Ensure tqdm works with pandas
tqdm.pandas()

# Initialize columns
filtered_dataframe_stage_2['Travelling mode'] = None
filtered_dataframe_stage_2['place_name_travel_mode'] = None

# Function to update Travelling mode and place_name_travel_mode based on multiple conditions
def update_travelling_mode(row):
    if pd.notna(row['name_airport_boundary']):
        return 'Air', row['name_airport_boundary']
    elif row['dist_to_airport(m)'] < 1500:
        return 'Air', row['name_airport_point']
    elif row['dist_to_rail(m)'] < 25:
        return 'Rail', row['name_rail']
    elif row['distance_to_road(m)'] < 25:
        if (row['highway'] == 'primary' and row['distance_to_road(m)'] < 25) or \
           (row['highway'] == 'secondary' and row['distance_to_road(m)'] < 5) or \
           (row['highway'] in ['tertiary', 'residential'] and row['distance_to_road(m)'] < 2):
            return 'Road', row['highway']
    return row['Travelling mode'], row['place_name_travel_mode']  # Keep previous values if no condition is met

# Apply the function with tqdm progress bar
filtered_dataframe_stage_2[['Travelling mode', 'place_name_travel_mode']] = filtered_dataframe_stage_2.progress_apply(
    lambda row: pd.Series(update_travelling_mode(row)), axis=1
)

# Verify the results
# print(gdf_intrastate[['Travelling mode', 'place_name_travel_mode']].head())


100%|██████████| 5780982/5780982 [06:34<00:00, 14658.75it/s]


In [135]:
pd.set_option('display.max_columns', None)
filtered_dataframe_stage_2[filtered_dataframe_stage_2['Travelling mode'].notna()]

maid            datetime  latitude  \
17       00007182-8140-450f-b91d-b4522543e0f6 2022-09-11 17:39:54   30.2275   
18       00007182-8140-450f-b91d-b4522543e0f6 2022-09-12 01:37:58   30.2275   
46       000228fd-5c6d-41bd-8aa6-a919a8f79a63 2022-09-01 01:22:22   29.0285   
47       000228fd-5c6d-41bd-8aa6-a919a8f79a63 2022-09-01 08:51:13   29.0285   
49       000228fd-5c6d-41bd-8aa6-a919a8f79a63 2022-09-01 10:36:28   29.0285   
...                                       ...                 ...       ...   
5780977  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 03:46:18   30.7314   
5780978  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:38:33   30.7316   
5780979  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 06:42:57   30.7316   
5780980  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:40:41   30.7316   
5780981  ffff2b2a-fcf3-4699-8dda-50e05ecfb98d 2022-09-16 16:43:31   30.7336   

         longitude  displacement     distance   Velocity        state  \
17         78.6929      0.481601  2937.474982   0.020612  Uttarakhand   
18         78.6929      0.000000  2937.474982   0.000000  Uttarakhand   
46         79.2900      0.011119     0.011119   3.639107  Uttarakhand   
47         79.2900      0.000000     0.011119   0.000000  Uttarakhand   
49         79.2900      0.011119     0.033358   0.006350  Uttarakhand   
...            ...           ...          ...        ...          ...   
5780977    79.0679    293.813191  7795.669911  20.190263  Uttarakhand   
5780978    79.0678      0.024206  7795.694117   0.008432  Uttarakhand   
5780979    79.0678      0.000000  7795.694117   0.000000  Uttarakhand   
5780980    79.0677      0.009558  7795.703675   0.000959  Uttarakhand   
5780981    79.0672      0.227467  7795.931142   4.816941  Uttarakhand   

                  district Nearest airport  Distance to airport (km)  \
17           Pauri Garhwal               0                       0.0   
18           Pauri Garhwal               0                       0.0   
46       Udham Singh Nagar               0                       0.0   
47       Udham Singh Nagar               0                       0.0   
49       Udham Singh Nagar               0                       0.0   
...                    ...             ...                       ...   
5780977        Rudraprayag               0                       0.0   
5780978        Rudraprayag               0                       0.0   
5780979        Rudraprayag               0                       0.0   
5780980        Rudraprayag               0                       0.0   
5780981        Rudraprayag               0                       0.0   

        origin_state    origin_district        date      time             y  \
17             Delhi          New Delhi  2022-09-11  17:39:54  8.760054e+06   
18             Delhi          New Delhi  2022-09-12  01:37:58  8.760054e+06   
46       Uttarakhand  Udham Singh Nagar  2022-09-01  01:22:22  8.826522e+06   
47       Uttarakhand  Udham Singh Nagar  2022-09-01  08:51:13  8.826522e+06   
49       Uttarakhand  Udham Singh Nagar  2022-09-01  10:36:28  8.826522e+06   
...              ...                ...         ...       ...           ...   
5780977  Uttarakhand        Rudraprayag  2022-09-16  03:46:18  8.801798e+06   
5780978  Uttarakhand        Rudraprayag  2022-09-16  06:38:33  8.801787e+06   
5780979  Uttarakhand        Rudraprayag  2022-09-16  06:42:57  8.801787e+06   
5780980  Uttarakhand        Rudraprayag  2022-09-16  16:40:41  8.801776e+06   
5780981  Uttarakhand        Rudraprayag  2022-09-16  16:43:31  8.801720e+06   

                    x position_group time_group  \
17       3.532826e+06     x_768y_495      t_211   
18       3.532826e+06     x_768y_495       t_19   
46       3.379274e+06     x_154y_761       t_16   
47       3.379274e+06     x_154y_761      t_106   
49       3.379274e+06     x_154y_761      t_127   
...               ...            ...        ...   
5780977  3.597915e+06    x_1029y_662       t

In [134]:
filtered_dataframe_stage_2.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Activity determination\Trajectory 2\step_7_UK_records_with_travel_modes.csv', index=False)